<a href="https://colab.research.google.com/github/mantoan-thi/AutoML/blob/main/Day_Trading_with_Machine_Learning_(witn_H2O_AutoML_Automatic_Machine_Learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalações

In [1]:
!pip install h2o

     |████████████████████████████████| 175.8 MB 33 kB/s 
  Created wheel for h2o: filename=h2o-3.34.0.1-py2.py3-none-any.whl size=175823552 sha256=3b8c162c4418c9877fabbe2762adb1e969acf0dd08040d9eb8775c6e72ee59b3
  Stored in directory: /root/.cache/pip/wheels/5c/7b/60/014a2b6d009793271276a9a41cb954659c8f23b8823ca21625
Successfully built h2o


In [2]:
!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 5.2 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=73d6d803e3007defbeae40a50ff41b33016cdeb3a104ca50b76e2fa1e98030df
  Stored in directory: /tmp/pip-ephem-wheel-cache-ot_h2b_t/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


# Importando as bibliotecas

In [18]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

# Treinando o modelo

In [27]:
def processar(dados, cls,tm):
  data = dados
  classe = cls
  tempo_maximo = tm
  h2o.init()
  # Identifica dinamicamente as colunas do arquivo csv
  colunas = data.columns.tolist()
  variaveis_independentes = [coluna for coluna in colunas if coluna != classe]

  # Divide os dados em treino e teste
  data = h2o.H2OFrame(data)
  treino, teste = data.split_frame(ratios=[.7])

  # Transforma a variavel dependente em fator
  treino[classe] = treino[classe].asfactor()
  teste[classe] = teste[classe].asfactor()

  # Auto ML
  # Busca o modelo valor gravado no atributo tempo_maximo em segundos. podemos em vez disso definir max_models
  modelo_automl = H2OAutoML(max_runtime_secs=tempo_maximo,sort_metric='AUC')
  #modelo_automl = H2OAutoML(max_models=20, seed=1)
  modelo_automl.train(y=classe, training_frame=treino)

  # Ranking dos melhores AutoML
  ranking = modelo_automl.leaderboard
  #ranking = ranking.as_data_frame()

  # Imprime todas as linhas em vez do padrão (10 linhas)
  ranking.head(rows=ranking.nrows)

  # O modelo líder é armazenado aqui
  modelo_automl.leader

  return ranking, modelo_automl,treino,teste

In [35]:
def prever(aml, dados_test):
  # Para gerar previsões em um conjunto de teste, você pode fazer previsões
  # diretamente no objeto `H2OAutoML` ou no modelo líder
  # objeto diretamente
  preds = aml.predict(dados_test)

  # ou:
  #preds = aml.leader.predict(dados_test)
  return preds

# Coletando e processando dados da Bolsa

importando as bibliotecas

In [21]:
import yfinance as yf #Instalando e importando a API no código

Coletando e processando

In [22]:
def coletar_dados(ativo):
    df = yf.download(ativo, start="2009-01-01", end="2021-04-30")
    df.reset_index(inplace=True)
    df['Tendencia_3d'] = (df['Close'].shift(-1)-df['Open'].shift(-1))+(df['Close'].shift(-2)-df['Open'].shift(-2))+(df['Close'].shift(-3)-df['Open'].shift(-3))
    df['Low_open'] = df.Low-df.Open
    df['High_open'] = df.High-df.Open
    df['Variação'] = df['Adj Close'].pct_change()
    df.dropna(inplace=True)
    df.drop(columns=['Date'],axis=1, inplace=True)
    df['Dir'] = df.apply(lambda x: 1 if x['Tendencia_3d'] > 0 else 0, axis=1)
    return df

Coletando dados do ITAU4

In [23]:
dados = coletar_dados('ITUB4.SA')

[*********************100%***********************]  1 of 1 completed


In [24]:
dados.shape

(3041, 11)

In [25]:
# Colunas
dados.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Tendencia_3d',
       'Low_open', 'High_open', 'Variação', 'Dir'],
      dtype='object')

In [28]:
# Iremos pegar a coluna Dir
cls = 'Dir'
ranking, modelo_automl,treino,teste = processar(dados,cls,60)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp88edevk_
  JVM stdout: /tmp/tmp88edevk_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp88edevk_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.1
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_unknownUser_fldfdr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


In [30]:
# Ranking
ranking

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_1_20210918_120528,1,0.0197752,1,0,0.0299883,0.0008993
XGBoost_2_AutoML_1_20210918_120528,1,0.00939692,1,0,0.0206541,0.000426593
DRF_1_AutoML_1_20210918_120528,0.999996,0.0448597,0.999996,0.000944287,0.0778598,0.00606215
StackedEnsemble_AllModels_1_AutoML_1_20210918_120528,0.999995,0.00394928,0.999995,0.000949668,0.0284622,0.000810099
StackedEnsemble_BestOfFamily_1_AutoML_1_20210918_120528,0.999994,0.00432177,0.999994,0.000949668,0.0302203,0.000913268
GBM_1_AutoML_1_20210918_120528,0.999993,0.00352925,0.999993,0.000949668,0.0318971,0.00101742
StackedEnsemble_BestOfFamily_2_AutoML_1_20210918_120528,0.999992,0.00432472,0.999992,0.000949668,0.0303652,0.000922046
GBM_3_AutoML_1_20210918_120528,0.999991,0.0237285,0.999991,0.000949668,0.0369441,0.00136486
GBM_2_AutoML_1_20210918_120528,0.999988,0.0287458,0.999988,0.000949668,0.0414855,0.00172105
GBM_4_AutoML_1_20210918_120528,0.999987,0.0475743,0.999987,0.000949668,0.056583,0.00320163


In [31]:
# Test
teste

Open,High,Low,Close,Adj Close,Volume,Tendencia_3d,Low_open,High_open,Variação,Dir
12.0459,12.5343,11.8637,12.4764,7.87314,1.29086e+07,-0.389112,-0.182137,0.488459,0.0411064,0
11.7354,11.9465,11.5905,11.8306,7.46563,7.25116e+06,-0.633342,-0.144881,0.211113,0.0059828,0
11.1849,11.396,10.7916,10.9034,6.88051,1.22583e+07,-1.20873,-0.39325,0.211114,-0.0365761,0
9.19378,9.76088,9.15238,9.63256,6.07856,1.0555e+07,0.0579519,-0.0413952,0.567108,0.022857,1
9.68637,9.86851,9.51665,9.57047,6.03938,9.55e+06,0.0289755,-0.169718,0.182137,-0.0064462,1
9.62428,9.70707,9.35521,9.43386,5.95318,9.507e+06,0.219392,-0.269066,0.0827904,-0.0142726,1
9.46284,9.71121,9.36349,9.38005,5.91922,8.08943e+06,0.71613,-0.0993471,0.248369,-0.0357451,1
10.6385,10.804,10.5639,10.6591,6.72638,8.72139e+06,-0.645759,-0.0745106,0.16558,-0.0152973,0
10.2204,10.2245,9.8147,9.89335,6.24313,1.7595e+07,-0.434644,-0.405667,0.00414085,-0.0590553,0
9.56633,9.616,9.35521,9.35521,5.90354,9.7365e+06,-0.413949,-0.211113,0.049674,0.00221723,0


In [36]:
prever(modelo_automl,teste)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.985313,0.0146872
0,0.985313,0.0146872
0,0.985313,0.0146872
1,0.0123569,0.987643
1,0.0123569,0.987643
1,0.0123569,0.987643
1,0.0123569,0.987643
0,0.985313,0.0146872
0,0.985313,0.0146872
0,0.985313,0.0146872


In [38]:
modelo_automl.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20210918_120528


Model Summary: 


,,number_of_trees
0,,50.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.00023629117763650053
RMSE: 0.015371765599191933
LogLoss: 0.013683210118538744
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 0.9999999999999999
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6791589260101318: 


,,0,1,Error,Rate
0,0,1059.0,0.0,0.0,(0.0/1059.0)
1,1,0.0,1053.0,0.0,(0.0/1053.0)
2,Total,1059.0,1053.0,0.0,(0.0/2112.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,auc,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,err,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,err_count,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,f0point5,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
5,f1,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,f2,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,lift_top_group,2.009219,0.094030,2.063415,1.922727,1.909502,2.131313,2.019139
8,logloss,0.019775,0.003837,0.015917,0.024835,0.021760,0.016014,0.020347
9,max_per_class_error,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2021-09-18 12:05:35,6.504 sec,0.0,0.500000,0.693147,0.5,0.49858,1.000000,0.50142
1,,2021-09-18 12:05:35,6.552 sec,5.0,0.104819,0.109361,1.0,1.00000,2.005698,0.00000
2,,2021-09-18 12:05:35,6.590 sec,10.0,0.026089,0.024170,1.0,1.00000,2.005698,0.00000
3,,2021-09-18 12:05:35,6.627 sec,15.0,0.015391,0.013683,1.0,1.00000,2.005698,0.00000
4,,2021-09-18 12:05:35,6.664 sec,20.0,0.015393,0.013682,1.0,1.00000,2.005698,0.00000
5,,2021-09-18 12:05:35,6.700 sec,25.0,0.015391,0.013683,1.0,1.00000,2.005698,0.00000
6,,2021-09-18 12:05:35,6.746 sec,30.0,0.015368,0.013683,1.0,1.00000,2.005698,0.00000
7,,2021-09-18 12:05:35,6.780 sec,35.0,0.015371,0.013683,1.0,1.00000,2.005698,0.00000
8,,2021-09-18 12:05:35,6.810 sec,40.0,0.015371,0.013683,1.0,1.00000,2.005698,0.00000
9,,2021-09-18 12:05:35,6.845 sec,45.0,0.015364,0.013684,1.0,1.00000,2.005698,0.00000



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Tendencia_3d,3306.690918,1.000000,0.997530
1,Adj Close,7.785137,0.002354,0.002349
2,Open,0.318770,0.000096,0.000096
3,High,0.084151,0.000025,0.000025
